In [4]:
#r "nuget:Microsoft.Diagnostics.NETCore.Client,0.2.61701"
using Microsoft.Diagnostics.NETCore.Client;
using System.Diagnostics;
using System.Linq;

var availableProcs = DiagnosticsClient.GetPublishedProcesses()
    .Select(x => Process.GetProcessById(x))
    .Select(p => new { name = p.ProcessName, id = p.Id, module = p.MainModule.FileName });

foreach (var p in availableProcs)
    display(p);

Installing package Microsoft.Diagnostics.NETCore.Client, version 0.2.61701.....

name,id,module
dotnet,37488,C:\Program Files\dotnet\dotnet.exe


name,id,module
dotnet-interactive,8512,C:\Users\josalem\.dotnet\tools\dotnet-interactive.exe


name,id,module
dotnet,7188,C:\Program Files\dotnet\dotnet.exe


name,id,module
ConsoleApp,48604,C:\git\scratch\ConsoleApp\bin\Debug\netcoreapp3.1\ConsoleApp.exe


In [10]:
#r "nuget:Microsoft.Diagnostics.Tracing.TraceEvent,2.0.49"
#r "nuget:Microsoft.Diagnostics.NETCore.Client,0.2.61701"

using Microsoft.Diagnostics.NETCore.Client;
using System.Diagnostics;
using Microsoft.Diagnostics.Tracing;
using Microsoft.Diagnostics.Tracing.Parsers;
using System.Threading.Tasks;

var client = new DiagnosticsClient(availableProcs.Where(p => p.name == "ConsoleApp").FirstOrDefault()?.id ?? -1);
var session = client.StartEventPipeSession(
    providers: new List<EventPipeProvider> 
    { 
        new EventPipeProvider("Microsoft-Windows-DotNETRuntime", System.Diagnostics.Tracing.EventLevel.Verbose, (long)(-1))
    }
);

var source = new EventPipeEventSource(session.EventStream);
source.Clr.All += (TraceEvent obj) =>
{
    display(obj.Dump());
};

var readTask = Task.Run(() => source.Process());

Task.Delay(TimeSpan.FromSeconds(1)).Wait();
// source.StopProcessing();
session.Stop();
display("Stopping");
readTask.Wait();
display("Stopped");

Installing package Microsoft.Diagnostics.Tracing.TraceEvent, version 2.0.49.....

Installing package Microsoft.Diagnostics.NETCore.Client, version 0.2.61701.....

<Event MSec=     "0.2902" PID="48604" PName="Process(48604)" TID="54996" EventName="GC/SuspendEEStart"
  TimeStamp="02/27/20 11:45:18.708290" ID="9" Version="1" Keywords="0x00000001" TimeStampQPC="11,856,920,487,143" QPCTime="0.100us"
  Level="Informational" ProviderName="Microsoft-Windows-DotNETRuntime" ProviderGuid="e13c0d23-ccbc-4e12-931b-d9cc2eee27e4" ClassicProvider="False" ProcessorNumber="0"
  Opcode="10" TaskGuid="044973cd-251f-4dff-a3e9-9d6307286b05" Channel="0" PointerSize="8"
  CPU="0" EventIndex="0" TemplateType="GCSuspendEETraceData">
  <Payload Length="10">
       0:   1  0  0  0  8  0  0  0 |  b  0                     ........ ..
  </Payload>
</Event>

<Event MSec=     "0.2963" PID="48604" PName="Process(48604)" TID="54996" EventName="GC/SuspendEEStop"
  TimeStamp="02/27/20 11:45:18.708296" ID="8" Version="1" Keywords="0x00000001" TimeStampQPC="11,856,920,487,204" QPCTime="0.100us"
  Level="Informational" ProviderName="Microsoft-Windows-DotNETRuntime" ProviderGuid="e13c0d23-ccbc-4e12-931b-d9cc2eee27e4" ClassicProvider="False" ProcessorNumber="0"
  Opcode="137" TaskGuid="044973cd-251f-4dff-a3e9-9d6307286b05" Channel="0" PointerSize="8"
  CPU="0" EventIndex="1" TemplateType="GCNoUserDataTraceData">
  <Payload Length="2">
       0:   b  0                   |                           ..
  </Payload>
</Event>

<Event MSec=     "0.3112" PID="48604" PName="Process(48604)" TID="54996" EventName="GC/Triggered"
  TimeStamp="02/27/20 11:45:18.708311" ID="35" Version="0" Keywords="0x00000001" TimeStampQPC="11,856,920,487,353" QPCTime="0.100us"
  Level="Informational" ProviderName="Microsoft-Windows-DotNETRuntime" ProviderGuid="e13c0d23-ccbc-4e12-931b-d9cc2eee27e4" ClassicProvider="False" ProcessorNumber="0"
  Opcode="35" TaskGuid="044973cd-251f-4dff-a3e9-9d6307286b05" Channel="0" PointerSize="8"
  CPU="0" EventIndex="2" TemplateType="GCTriggeredTraceData">
  <Payload Length="6">
       0:   1  0  0  0  b  0       |                           ......
  </Payload>
</Event>

...

<Event MSec=     "0.8761" PID="48604" PName="Process(48604)" TID="54996" EventName="GC/RestartEEStart"
  TimeStamp="02/27/20 11:45:18.708876" ID="7" Version="1" Keywords="0x00000001" TimeStampQPC="11,856,920,493,002" QPCTime="0.100us"
  Level="Informational" ProviderName="Microsoft-Windows-DotNETRuntime" ProviderGuid="e13c0d23-ccbc-4e12-931b-d9cc2eee27e4" ClassicProvider="False" ProcessorNumber="0"
  Opcode="136" TaskGuid="044973cd-251f-4dff-a3e9-9d6307286b05" Channel="0" PointerSize="8"
  CPU="0" EventIndex="34" TemplateType="GCNoUserDataTraceData">
  <Payload Length="2">
       0:   b  0                   |                           ..
  </Payload>
</Event>

<Event MSec=     "0.8807" PID="48604" PName="Process(48604)" TID="54996" EventName="GC/RestartEEStop"
  TimeStamp="02/27/20 11:45:18.708880" ID="3" Version="1" Keywords="0x00000001" TimeStampQPC="11,856,920,493,048" QPCTime="0.100us"
  Level="Informational" ProviderName="Microsoft-Windows-DotNETRuntime" ProviderGuid="e13c0d23-ccbc-4e12-931b-d9cc2eee27e4" ClassicProvider="False" ProcessorNumber="0"
  Opcode="132" TaskGuid="044973cd-251f-4dff-a3e9-9d6307286b05" Channel="0" PointerSize="8"
  CPU="0" EventIndex="35" TemplateType="GCNoUserDataTraceData">
  <Payload Length="2">
       0:   b  0                   |                           ..
  </Payload>
</Event>

<Event MSec=     "0.9753" PID="48604" PName="Process(48604)" TID="12508" EventName="AppDomainResourceManagement/ThreadCreated"
  TimeStamp="02/27/20 11:45:18.708975" ID="85" Version="0" Keywords="0x00010800" TimeStampQPC="11,856,920,493,994" QPCTime="0.100us"
  Level="Informational" ProviderName="Microsoft-Windows-DotNETRuntime" ProviderGuid="e13c0d23-ccbc-4e12-931b-d9cc2eee27e4" ClassicProvider="False" ProcessorNumber="0"
  Opcode="50" TaskGuid="88e83959-6185-4e0b-95b8-0e4a35df6122" Channel="0" PointerSize="8"
  CPU="0" EventIndex="36" TemplateType="ThreadCreatedTraceData">
  <Payload Length="30">
       0:  d0 73 f7 e0 e1  1  0  0 | 80 64 f4 e0 e1  1  0  0   .s...... .d......
      10:   0  0  0  0  4  0  0  0 | dc 30  0  0  b  0         ........ .0....
  </Payload>
</Event>

Stopping

Stopped